<a href="https://colab.research.google.com/github/romeii/AgentGPT/blob/main/RAG_Demo_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Demo with Google Gemini + FAISS

This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) system that:
1. Converts PDF documents to text
2. Stores them in FAISS vector database (simple & fast)
3. Retrieves relevant context for questions
4. Uses Google Gemini to generate answers with citations

## 1. Install Dependencies

In [ ]:
!pip install google-generativeai pypdf langchain langchain-community faiss-cpu

## 2. Configure Google Gemini API

In [2]:
import os
import google.generativeai as genai
from google.colab import userdata

# Get API key from Colab Secrets
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY_2')
genai.configure(api_key=GEMINI_API_KEY)

## 3. Define Gemini Call Function

In [21]:
def call_gemini(question, contexts, model_name="models/gemini-2.5-flash"):
    """Call Gemini with question and retrieved contexts."""
    # Build context block
    context_block = ""
    for i, (chunk, source) in enumerate(contexts, start=1):
        context_block += f"[{i}] {chunk}\n(Source: {source})\n\n"

    prompt = f"""
You are a helpful assistant. Answer ONLY using the provided sources.
Cite them inline using [number] format. If the answer is not present, say
you don't know.

Question: {question}

Sources:
{context_block}
"""

    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)

    return response.text

## 4. Convert PDFs to Text

Upload your PDF files to Colab, then run this cell to convert them to text.

In [11]:
from pypdf import PdfReader

# Create docs directory
os.makedirs("docs", exist_ok=True)

# Convert all PDFs in /content to text
pdf_count = 0
for filename in os.listdir("/content"):
    if filename.lower().endswith(".pdf"):
        pdf_path = f"/content/{filename}"
        txt_path = f"/content/docs/{filename}.txt"

        with open(txt_path, "w", encoding="utf-8") as out:
            reader = PdfReader(pdf_path)
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    out.write(text + "\n")
        pdf_count += 1

print(f"✅ Converted {pdf_count} PDF(s) to text")
print("Files in docs/:", os.listdir("/content/docs"))

✅ Converted 4 PDF(s) to text
Files in docs/: ['2025-PL-Media-Kit-F.pdf.txt', 'Plate_Social Media Rates.pdf.txt', 'Plate_White Label Marketing Journey.pdf.txt', 'Plate Brand Guidelines 2023.pdf.txt']


## 5. Load and Chunk Documents

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Read all text files
documents = []
for filename in os.listdir("/content/docs"):
    if filename.endswith(".txt"):
        filepath = f"/content/docs/{filename}"
        with open(filepath, "r", encoding="utf-8") as f:
            text = f.read()
            documents.append({"text": text, "source": filename})

print(f"Loaded {len(documents)} document(s)")

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = []
for doc in documents:
    splits = text_splitter.split_text(doc["text"])
    for split in splits:
        chunks.append({"text": split, "source": doc["source"]})

print(f"Split into {len(chunks)} chunks")

Loaded 4 document(s)
Split into 59 chunks


## 6. Create FAISS Vector Store

In [13]:
import numpy as np
import faiss

# Create embeddings for all chunks
print("Creating embeddings...")
embeddings = []

for i, chunk in enumerate(chunks):
    result = genai.embed_content(
        model="models/embedding-001",
        content=chunk["text"],
        task_type="retrieval_document"
    )
    embeddings.append(result['embedding'])

    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1}/{len(chunks)} chunks")

# Convert to numpy array
embeddings_array = np.array(embeddings).astype('float32')

# Create FAISS index
dimension = embeddings_array.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_array)

print(f"\n✅ FAISS index created with {index.ntotal} vectors")

Creating embeddings...
Processed 10/59 chunks
Processed 20/59 chunks
Processed 30/59 chunks
Processed 40/59 chunks
Processed 50/59 chunks

✅ FAISS index created with 59 vectors


## 7. Query Function

In [14]:
def query_rag(question, k=3):
    """Query the RAG system with a question."""

    # Create embedding for the question
    question_embedding = genai.embed_content(
        model="models/embedding-001",
        content=question,
        task_type="retrieval_query"
    )['embedding']

    # Convert to numpy array
    query_vector = np.array([question_embedding]).astype('float32')

    # Search FAISS index
    distances, indices = index.search(query_vector, k)

    # Get the relevant chunks
    contexts = []
    for idx in indices[0]:
        chunk = chunks[idx]
        contexts.append((chunk["text"], chunk["source"]))

    # Get answer from Gemini
    answer = call_gemini(question, contexts)

    return answer

## 8. Test Query

In [ ]:
# Example usage
question = "What are the key topics covered in these documents?"
answer = query_rag(question)
print(f"Question: {question}\n")
print(f"Answer: {answer}")

## 9. Interactive Query Cell

Use this cell to ask your own questions:

In [ ]:
# Ask your own question
my_question = "What is Plate"
answer = query_rag(my_question)
print(f"Question: {my_question}\n")
print(f"Answer: {answer}")